In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
# 데이터 불러오기
base_path = os.getcwd()
dir_path = base_path + '/*.csv' # csv 파일이 저장되어있는 폴더 경로
csv_paths = glob.glob(dir_path) # 폴더 내에 모든 csv 파일 경로

df_sellers = pd.read_csv(csv_paths[0])
df_mql = pd.read_csv(csv_paths[1])
df_translation = pd.read_csv(csv_paths[2])
df_orders = pd.read_csv(csv_paths[3])
df_order_items = pd.read_csv(csv_paths[4])
df_customers = pd.read_csv(csv_paths[5])
df_geoloc = pd.read_csv(csv_paths[6])
df_payments = pd.read_csv(csv_paths[7])
df_closed_deals = pd.read_csv(csv_paths[8])
df_reviews = pd.read_csv(csv_paths[9])
df_product = pd.read_csv(csv_paths[10])

### 1. 결측치 및 불필요한 컬럼 처리

#### 1. closed_deals

In [3]:
df_closed_deals.head()

,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,NaN,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


In [4]:
# 불필요한 컬럼 삭제
df_closed_deals.drop(['has_company', 'has_gtin', 'average_stock', 'declared_product_catalog_size'], axis=1, inplace=True)

# lead_behaviour_profile 결측치는 ‘unknown’으로 대체
df_closed_deals.lead_behaviour_profile.fillna('unknown', inplace=True)

# lead_type business_segment business_type 결측치는 최빈값으로 대체
for col in ['lead_type', 'business_segment', 'business_type']:
    df_closed_deals[col].fillna(df_closed_deals[col].mode()[0], inplace=True)

In [5]:
df_closed_deals.isnull().sum()

mql_id                      0
seller_id                   0
sdr_id                      0
sr_id                       0
won_date                    0
business_segment            0
lead_type                   0
lead_behaviour_profile      0
business_type               0
declared_monthly_revenue    0
dtype: int64

#### 2. mql

In [6]:
df_mql.head()

,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [7]:
df_mql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [8]:
df_mql.origin.value_counts()

organic_search       2296
paid_search          1586
social               1350
unknown              1099
direct_traffic        499
email                 493
referral              284
other                 150
display               118
other_publicities      65
Name: origin, dtype: int64

In [9]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [10]:
# origin 결측치 unknown으로 대체
df_mql.origin.fillna('unknown', inplace=True)

In [11]:
df_mql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              8000 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


#### 3. product catogory 해석

In [4]:
df_product.merge(df_translation, on='product_category_name', how='left')

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares
...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,furniture_decor
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,construction_tools_lights
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,bed_bath_table
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,computers_accessories


In [5]:
# df_translation을 활용하여 product_category_name 번역
df_product = df_product.merge(df_translation, on='product_category_name', how='left')

# 'product_category_name_english' 삭제, 'product_category_name' 컬럼 사용
df_product.drop('product_category_name', axis=1, inplace=True)
df_product.rename(columns={'product_category_name_english': 'product_category_name'}, inplace=True)

# 컬럼순서 재배열
columns_prod = [
    'product_id',
    'product_category_name',
    'product_name_lenght',
    'product_description_lenght',
    'product_photos_qty',
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]

df_product = df_product[columns_prod]
df_product.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


#### 2. mql 테이블(mql, closed_deals)

In [15]:
lead = pd.merge(df_mql, df_closed_deals, how='left', on='mql_id')
lead.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mql_id                    8000 non-null   object 
 1   first_contact_date        8000 non-null   object 
 2   landing_page_id           8000 non-null   object 
 3   origin                    8000 non-null   object 
 4   seller_id                 842 non-null    object 
 5   sdr_id                    842 non-null    object 
 6   sr_id                     842 non-null    object 
 7   won_date                  842 non-null    object 
 8   business_segment          842 non-null    object 
 9   lead_type                 842 non-null    object 
 10  lead_behaviour_profile    842 non-null    object 
 11  business_type             842 non-null    object 
 12  declared_monthly_revenue  842 non-null    float64
dtypes: float64(1), object(12)
memory usage: 875.0+ KB


In [16]:
# lead.head()

### 2. id를 int 형식으로 바꾸기 (db 적재시 저장공간 효율화)

In [17]:
df_mql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              8000 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [19]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


#### 바꿀 id 목록
df_orders : order_id, customer_id <br>
df_payments : order_id <br>
df_customers : customer_id, customer_unique_id <br>
df_order_items : order_id, product_id, seller_id <br>
df_product : product_id <br>
df_sellers : seller_id <br>
df_closed_deals : mql_id, seller_id, sdr_id, sr_id <br>
df_mql : mql_id <br>
df_reviews : review_id, order_id

In [20]:
def id_mapping(*dfs, id):
    id_unique = set()
    
    for df in dfs:
        id_unique |= set(df[id]) - {np.nan}
    
    mapping = dict(zip(id_unique, np.arange(len(id_unique))))
    
    for df in dfs:
        df[id] = df[id].map(mapping)

In [22]:
df_orders_mapped = df_orders.copy()
df_payments_mapped = df_payments.copy()
df_customers_mapped = df_customers.copy()
df_order_items_mapped = df_order_items.copy()
df_product_mapped = df_product.copy()
df_sellers_mapped = df_sellers.copy()
df_reviews_mapped = df_reviews.copy()
df_closed_deals_mapped = df_closed_deals.copy()
df_mql_mapped = df_mql.copy()

In [23]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [24]:
id_mapping(df_orders_mapped, df_payments_mapped, df_order_items_mapped, df_reviews_mapped, id='order_id')
id_mapping(df_customers_mapped, df_orders_mapped, id='customer_id')
id_mapping(df_customers_mapped, id='customer_unique_id')
id_mapping(df_product_mapped, df_order_items_mapped, id='product_id')
id_mapping(df_sellers_mapped, df_order_items_mapped, df_closed_deals_mapped, id='seller_id')
id_mapping(df_mql_mapped, df_closed_deals_mapped, id='mql_id')
id_mapping(df_closed_deals_mapped, id='sdr_id')
id_mapping(df_closed_deals_mapped, id='sr_id')
id_mapping(df_mql_mapped, id='landing_page_id')
id_mapping(df_reviews_mapped, id='review_id')

### 3. 데이터타입 변환

#### 1. 시계열 변환

In [25]:
def to_date (df, column):
    for col in column:
        df[col] = pd.to_datetime(df[col])

In [26]:
to_date(df_orders_mapped, ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])
to_date(df_order_items_mapped, ['shipping_limit_date'])
to_date(df_mql_mapped, ['first_contact_date'])
to_date(df_closed_deals_mapped, ['won_date'])
to_date(df_reviews_mapped, ['review_creation_date', 'review_answer_timestamp'])


#### 2. int 변환

In [27]:
df_order_items_mapped.order_item_id = df_order_items_mapped.order_item_id.astype('int')

### db 적재

In [28]:
# 데이터를 csv파일로 전환하지 않고 바로 DB에 연결시킬 수 있는 함수
def data_store(dataframe,table_name):

    from sqlalchemy import create_engine

    host = 'postgres.c4brhfvhrm5g.ap-northeast-2.rds.amazonaws.com'
    dbname = 'postgres' 
    user = 'postgres' 
    password = 'postgres123'
    port = 5432

    # SQLAlchemy 엔진 생성
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

    # 데이터프레임을 데이터베이스에 적재
    dataframe.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

In [29]:
# 비밀번호 : postgres123
# username : postgres
# database : postgres
# Host : postgres.c4brhfvhrm5g.ap-northeast-2.rds.amazonaws.com
# port : 5432

In [30]:
# 컬럼명 오타 변환
df_product_mapped.rename({'product_name_lenght':'product_name_length', 'product_description_lenght':'product_description_length'}, axis=1, inplace=True)

In [31]:
data_store(df_orders_mapped, 'order')
data_store(df_payments_mapped, 'payments')
data_store(df_customers_mapped, 'customers')
data_store(df_order_items_mapped, 'order_items')
data_store(df_product_mapped, 'products')
data_store(df_sellers_mapped, 'sellers')
data_store(df_reviews_mapped, 'reviews')
data_store(df_closed_deals_mapped, 'closed_deals')
data_store(df_mql_mapped, 'mql')
data_store(df_geoloc, 'geoloc')